In [63]:
import pandas as pd
import numpy as np

In [2]:
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [114]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [6]:
symbols = ['btc', 'eth', 'aapl', 'spx']

In [7]:
'C:/Users/Comarch/Desktop/miko_work/'

'C:/Users/Comarch/Desktop/miko_work/'

In [8]:
import os

In [164]:
from xgboost import XGBClassifier

In [170]:
for file in os.listdir('C:/Users/Comarch/Desktop/miko_work/pred'):
    symbol = file.split('.')[0].split('_')[4]
    if symbol=='btc':
        data = pd.read_csv('C:/Users/Comarch/Desktop/miko_work/pred/' + file, index_col=0, parse_dates=True)
        changes = pd.read_csv('C:/Users/Comarch/Desktop/miko_work/changes/' + symbol + '_changes.csv', index_col=0, parse_dates=True)
        roll = pd.read_csv('C:/Users/Comarch/Desktop/miko_work/roll/' + symbol + '_roll.csv', index_col=0, parse_dates=True)
#         volume = pd.read_csv('C:/Users/Comarch/Desktop/miko_work/pred/pred_stw_vol_brt_' + symbol + '.csv', index_col=0, parse_dates=True)
#         data = pd.concat([data.iloc[:, 0], volume.iloc[:, 0]], axis=1)
        
        data_train = data[data.index < pd.to_datetime('2019-05-01')]
        data_test = data[data.index >= pd.to_datetime('2019-05-01')]
        changes_train = changes[changes.index < pd.to_datetime('2019-05-01')]
        changes_test = changes[changes.index >= pd.to_datetime('2019-05-01')]
        roll_train = roll[roll.index < pd.to_datetime('2019-05-01')]
        roll_test = roll[roll.index >= pd.to_datetime('2019-05-01')]
        
        tg_train = TimeseriesGenerator(data_train.values, changes_train.values, 3, batch_size=len(data_train))
        x_train, y_train = tg_train[0]
        x_train = x_train.reshape(-1, x_train.shape[1] * x_train.shape[2])
        y_train = y_train.reshape(-1)
        
        tg_test = TimeseriesGenerator(data_test.values, changes_test.values, 3, batch_size=len(data_test))
        x_test, y_test = tg_test[0]
        x_test = x_test.reshape(-1, x_test.shape[1] * x_test.shape[2])
        y_test = y_test.reshape(-1)
        
        x_train = np.append(x_train, roll_train[2:-1], axis=1)
        x_test = np.append(x_test, roll_test[2:-1], axis=1)
        
        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)
        
        model = LogisticRegression()
        model.fit(x_train, y_train)
        prob = model.predict_proba(x_test)[:, 1]
        acc = roc_auc_score(y_test, prob)
        print(file, str(acc))

pred_sto_all_si1_btc.csv 0.6255482456140351
pred_sto_all_si2_btc.csv 0.6337719298245614
pred_sto_all_si3_btc.csv 0.5932017543859649
pred_sto_fbp_si1_btc.csv 0.6129385964912281
pred_sto_fbp_si2_btc.csv 0.6129385964912281
pred_sto_fbp_si3_btc.csv 0.6080043859649122
pred_sto_rea_all_btc.csv 0.5427631578947368
pred_sto_rea_fbp_btc.csv 0.5904605263157894
pred_sto_rea_twt_btc.csv 0.5021929824561404
pred_sto_twt_si1_btc.csv 0.5899122807017544
pred_sto_twt_si2_btc.csv 0.5921052631578947
pred_sto_twt_si3_btc.csv 0.5959429824561403
pred_sto_vol_all_btc.csv 0.6348684210526315
pred_sto_vol_fbp_btc.csv 0.5855263157894737
pred_sto_vol_twt_btc.csv 0.569078947368421
pred_stw_brt_si1_btc.csv 0.6282894736842105
pred_stw_brt_si2_btc.csv 0.5970394736842105
pred_stw_brt_si3_btc.csv 0.638157894736842
pred_stw_dcl_si1_btc.csv 0.6266447368421053
pred_stw_dcl_si2_btc.csv 0.5948464912280701
pred_stw_dcl_si3_btc.csv 0.6425438596491228
pred_stw_vol_brt_btc.csv 0.5241228070175438
pred_stw_vol_dcl_btc.csv 0.5345394